In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("./src/")
sys.path.append("../dift/src/models")

In [ ]:
from utils import load_frames
frames, frames_fpaths = load_frames('../test_videos/frames/bmx_short/color/')
print(f'loaded {len(frames)} frames')

In [ ]:
from src.keypoint_extractor import KeypointExtractor
extractor = KeypointExtractor(device='cuda:1')

# Run extractor functions separately

In [ ]:
frames = frames[:10]

In [ ]:
features = extractor.img_to_features(frames, 'bmx biker in a skate park')

In [ ]:
src_ft = features[0].unsqueeze(0)
target_features = features[6:7]
correspondences = extractor.get_correspondence(src_ft, target_features, (100, 100))

In [ ]:
correspondences

# Video Tracking

In [ ]:
from performance import PerformanceManager

In [ ]:
perf_manager = PerformanceManager()
keypoints = extractor.track_keypoints(frames[:100], prompt='biker in a skate park', source_frame_idx=0, grid_size=10, perf_manager=perf_manager)

In [ ]:
keypoints

In [ ]:
from utils import draw_keypoint_on_frame
frame = frames[9].copy()
for keypoint in keypoints[9]:
    frame = draw_keypoint_on_frame(frame, keypoint, dynamic_color=True, radius=1.5)
frame

In [ ]:
perf_manager.print_summary()

# Test video saving

In [ ]:
import cv2
import numpy as np
from PIL import Image
import subprocess
import os

def save_frames_as_video(frames, fpath, fps=24):
    """
    Save a list of PIL Image frames as an mp4 video and then convert it to webm using FFmpeg.
    
    Parameters:
    - frames: List of PIL Image objects
    - fpath: Path where the mp4 video will be saved
    - fps: Desired frames per second for the output video (default: 24)
    
    Returns:
    None
    """
    
    # Check if there are frames
    if not frames:
        raise ValueError("No frames provided")
    
    # Convert the first frame to numpy to get the dimensions
    frame_np = np.array(frames[0])
    h, w, layers = frame_np.shape
    size = (w, h)
    
    _, ext = os.path.splitext(fpath)
    if ext != '.webm':
        raise ValueError("Output file must be a webm video")

    temp_fpath = fpath.replace('.webm', '.mp4')
    
    # Define the codec using VideoWriter_fourcc and create a VideoWriter object
    out = cv2.VideoWriter(temp_fpath, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    
    for frame in frames:
        # Convert PIL Image to NumPy array
        frame_np = np.array(frame)
        
        # Convert RGB to BGR format (as OpenCV works with BGR for writing videos)
        frame_bgr = cv2.cvtColor(frame_np, cv2.COLOR_RGB2BGR)
        
        # Write the frame to the video
        out.write(frame_bgr)
    
    # Release the VideoWriter
    out.release()

    print(f'Video saved to {temp_fpath}; converting to {fpath}')
    
    # Construct the FFmpeg command
    cmd = ['/usr/bin/ffmpeg', '-y', '-i', temp_fpath, '-c:v', 'libvpx-vp9', '-b:v', '1M', '-c:a', 'libopus', fpath]



    # Run the FFmpeg command
    subprocess.run(cmd, check=True)
    # remove mp4 temp file
    os.remove(temp_fpath)

In [ ]:
save_frames_as_video(frames, 'test.webm')